In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import numpy as np


In [3]:
df = pd.read_csv('\\Users\Mohamed\MS BGD\\toolkit Big Data\hackathon\R2015AL\R201501.CSV', delimiter=";", encoding = "ISO-8859-1")

C:\Users\Mohamed\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [83]:
dfclean = df.drop(["cpam",  "dpt", "region", "l_serie", "SERIE", "l_prs_nat",\
 "asu_nat", "l_cpl_cod", "ben_qlt", "l_ben_qlt", "l_pre_spe1", "pre_spe", \
 "pre_stj1", "exe_spe", "l_exe_spe", "exe_spe1","exe_stj1", "top_slm",\
  "l_top_slm", "l_exe_stj1", "prs_nat", "sns_date", "pre_spe1", "act_dnb",\
              "act_coe", "l_pre_stj1", "l_exe_stj1", "l_exe_spe1", "REM_TAU"], axis=1)
print(dfclean.columns)
dfclean["l_pre_spe"] = dfclean["l_pre_spe"].apply(lambda x : x[3:].replace("TOTAL ", "").strip())
dfclean["l_region"] = dfclean["l_region"].apply(lambda x : x[3:].strip())

dfclean["l_region"] = dfclean["l_region"].apply(lambda x : "Alsace-Champagne-Ardenne-Lorraine" if "LORRAINE" in x or "ALSACE" in x or "CHAMPAGNE ARDENNE" in x else x)
dfclean["l_region"] = dfclean["l_region"].apply(lambda x : "Normandie" if "NORMANDIE" in x else x)
dfclean["l_region"] = dfclean["l_region"].apply(lambda x : "Aquitaine-Limousin-Poitou-Charentes" if "AQUITAINE" in x or "LIMOUSIN" in x or "POITOU CHARENTES" in x else x)
dfclean["l_region"] = dfclean["l_region"].apply(lambda x : "Auvergne-Rhône-Alpes" if "AUVERGNE" in x or "RHONES ALPES" in x else x)
dfclean["l_region"] = dfclean["l_region"].apply(lambda x : "Bourgogne-Franche-Comté" if "BOURGOGNE" in x or "FRANCHE COMTE" in x else x)
dfclean["l_region"] = dfclean["l_region"].apply(lambda x : "Centre-Val de Loire" if "CENTRE" in x else x)
dfclean["l_region"] = dfclean["l_region"].apply(lambda x : "Languedoc-Roussillon-Midi-Pyrénées" if "LANGUEDOC ROUSSILLON" in x or "MIDI PYRENEES" in x else x)
dfclean["l_region"] = dfclean["l_region"].apply(lambda x : "Nord-Pas-de-Calais-Picardie" if "NORD PAS DE CALAIS" in x or "PICARDIE" in x else x)
dfclean["l_region"] = dfclean["l_region"].apply(lambda x : x.replace("-", " ").upper())
dfclean = dfclean.rename(columns = {'l_region' : "REGION"})
dfclean = dfclean.rename(columns = {'l_pre_spe' : "SPECIALITE"})

print(dfclean["REGION"].unique())
print(dfclean["SPECIALITE"].unique())

Index(['l_cpam', 'l_dpt', 'l_region', 'l_asu_nat', 'cpl_cod', 'l_pre_spe',
       'rem_mon', 'rec_mon', 'dep_mon'],
      dtype='object')
['NORD PAS DE CALAIS PICARDIE' 'LANGUEDOC ROUSSILLON MIDI PYRÉNÉES'
 'CENTRE VAL DE LOIRE' 'ALSACE CHAMPAGNE ARDENNE LORRAINE' 'ILE DE FRANCE'
 'AQUITAINE LIMOUSIN POITOU CHARENTES' 'PAYS DE LA LOIRE' 'RHONE ALPES'
 'BOURGOGNE FRANCHE COMTÉ' 'NORMANDIE' 'BRETAGNE' 'GUADELOUPE'
 "PROVENCE ALPES COTE D'AZUR" 'MARTINIQUE' 'CORSE' 'AUVERGNE RHÔNE ALPES'
 'GUYANE' 'REUNION' 'MAYOTTE']
['Omnipraticiens' 'Spécialité Inconnue' 'Chirurgie'
 'Pathologie cardio-vasculaire' 'Anesthésie-réanimation chirurgicale'
 'Ophtalmologie' 'Dermatologie et vénérologie' 'Oto-rhino laryngologie'
 'Gastro-entérologie et hépatologie' 'Stomatologie' 'Médecine Interne'
 'Rhumatologie' 'Radiologie' 'Gynécologie' 'Pneumologie' 'Endocrinologie'
 'Pédiatrie' 'Néphrologie' 'Anatomie-cytologie-pathologique' 'Gériatrie'
 'Médecine physique et de réanimation' 'Santé publique et Médecine 

In [84]:
dfclean["dep_mon"] = dfclean["dep_mon"].apply(lambda x : float(x.replace(".", "").replace(",", ".")))
groupedDfClean = dfclean.groupby(["REGION", "SPECIALITE"], as_index=False)["dep_mon"].sum()

print(groupedDfClean.head(5))


                              REGION                           SPECIALITE  \
0  ALSACE CHAMPAGNE ARDENNE LORRAINE      Anatomie-cytologie-pathologique   
1  ALSACE CHAMPAGNE ARDENNE LORRAINE  Anesthésie-réanimation chirurgicale   
2  ALSACE CHAMPAGNE ARDENNE LORRAINE                            Chirurgie   
3  ALSACE CHAMPAGNE ARDENNE LORRAINE                   Chirurgie dentaire   
4  ALSACE CHAMPAGNE ARDENNE LORRAINE      Chirurgie dentaire spécialistes   

       dep_mon  
0     43521.87  
1   1046063.44  
2   3470705.29  
3  20863020.92  
4   5474843.95  


In [118]:
dfMed = pd.read_csv("C:\\Users\Mohamed\MS BGD\\toolkit Big Data\hackathon\EffectifMedecins.CSV", delimiter= ",", encoding = "ISO-8859-1", skiprows =3)
dfMed = dfMed.rename(columns={'Unnamed: 5' : 'Effectif'})
dfMed = dfMed[dfMed.SEXE.str.contains("confondus") == False]
dfMed = dfMed[dfMed.AGE.str.contains("confondus") == False]
dfMed = dfMed[dfMed['REGION ACTIVITE'] != "FRANCE ENTIERE"]
dfMed = dfMed[dfMed['REGION ACTIVITE'] != "FRANCE Métropolitaine"]
dfMed = dfMed[dfMed.SPECIALITE.str.contains("Ensemble") == False]
dfMed = dfMed[dfMed.SPECIALITE.str.contains("Spécialistes") == False]
dfMed = dfMed[dfMed['ANNEE'] == 2015]

dfMed["SPECIALITE"] = dfMed["SPECIALITE"].apply(lambda x : "Chirurgie" if "Chirurgie" in x else x)
dfMed["SPECIALITE"] = dfMed["SPECIALITE"].apply(lambda x : "Gynécologie" if "Gynécologie" in x else x)
dfMed["SPECIALITE"] = dfMed["SPECIALITE"].apply(lambda x : "Pathologie cardio-vasculaire" if "cardio" in x else x)
dfMed["SPECIALITE"] = dfMed["SPECIALITE"].apply(lambda x : "Anesthésie-réanimation chirurgicale" if "Anesthésie" in x else x)
dfMed["SPECIALITE"] = dfMed["SPECIALITE"].apply(lambda x : "Oto-rhino laryngologie" if "ORL" in x else x)
dfMed["SPECIALITE"] = dfMed["SPECIALITE"].apply(lambda x : "Stomatologie" if "stomato" in x else x)
dfMed["SPECIALITE"] = dfMed["SPECIALITE"].apply(lambda x : "Radiologie" if "Radio" in x else x)
dfMed["SPECIALITE"] = dfMed["SPECIALITE"].apply(lambda x : "Endocrinologie" if "Endocrinolo" in x else x)
dfMed["SPECIALITE"] = dfMed["SPECIALITE"].apply(lambda x : "Anatomie-cytologie-pathologique" if "Anatomie" in x else x)
dfMed["SPECIALITE"] = dfMed["SPECIALITE"].apply(lambda x : "Médecin biologiste" if "Biologie" in x else x)
dfMed["REGION ACTIVITE"] = dfMed["REGION ACTIVITE"].apply(lambda x : x.replace("-", " ").upper())

dfMed = dfMed.rename(columns = {"REGION ACTIVITE" : "REGION"})
print(dfMed.columns)
new_index = (np.arange(len(dfMed["SEXE"])))
dfMed = dfMed.set_index(new_index)

groupedMed = dfMed.groupby(['REGION','SPECIALITE'], as_index=False)
groupedMed = groupedMed["Effectif"].sum()

print(grouped.head(5))

Index(['SEXE', 'AGE', 'ANNEE', 'REGION', 'SPECIALITE', 'Effectif'], dtype='object')
                     REGION ACTIVITE                           SPECIALITE  \
0  ALSACE CHAMPAGNE ARDENNE LORRAINE      Anatomie-cytologie-pathologique   
1  ALSACE CHAMPAGNE ARDENNE LORRAINE  Anesthésie-réanimation chirurgicale   
2  ALSACE CHAMPAGNE ARDENNE LORRAINE  Cardiologie et maladies vasculaires   
3  ALSACE CHAMPAGNE ARDENNE LORRAINE                            Chirurgie   
4  ALSACE CHAMPAGNE ARDENNE LORRAINE         Dermatologie et vénéréologie   

   Effectif  
0       107  
1       936  
2       567  
3       965  
4       306  


In [172]:
import unicodedata

joinedDf = pd.merge(groupedDfClean, groupedMed, how='inner', on=['REGION','SPECIALITE'])

densitePop = pd.read_excel("C:\\Users\Mohamed\MS BGD\\toolkit Big Data\hackathon\estim-pop-reg-sexe-gca-1975-2015.xlsx", sheetname="2015", skiprows=4)
densitePop = densitePop.rename(columns = {"Unnamed: 0" : "REGION"})
densitePop = densitePop.drop(densitePop.columns[7:-1], axis=1)
densitePop = densitePop.drop(densitePop.columns[-1], axis=1)
densitePop["REGION"] = densitePop["REGION"].apply(lambda x : str(x).replace("-", " ").replace("é", "e").replace("ô", "o").replace("Î", "i").upper())

joinedWPopReg = pd.merge(joinedDf, densitePop, how = 'inner', on = "REGION")

joinedDfByRegion = joinedWPopReg.groupby('REGION', as_index=False).sum()
joinedDfBySpec = joinedDf.groupby('SPECIALITE', as_index=False).sum()
joinedDfBySpec["Depassement/eff"] = joinedDfBySpec["dep_mon"]/joinedDfBySpec["Effectif"]
joinedDfByRegion["Depassement/eff"] = joinedDfByRegion["dep_mon"]/joinedDfByRegion["Effectif"]
joinedDfByRegion = joinedDfByRegion.set_index(["REGION"])
joinedDfBySpec = joinedDfBySpec.set_index(["SPECIALITE"])
print(joinedDfByRegion.sort_values(['Depassement/eff']))
print(joinedDfBySpec.sort_values(['Depassement/eff'])) 
print(np.corrcoef(joinedDfByRegion["dep_mon"], joinedDfByRegion["Total"]))
 
parisOphtalmo = (joinedDf[(joinedDf["REGION"] == "ILE DE FRANCE") & (joinedDf["SPECIALITE"] == "Ophtalmologie")])
parisOphtalmo["Depassement/eff"] = parisOphtalmo["dep_mon"]/parisOphtalmo["Effectif"]
print(parisOphtalmo)


['ALSACE CHAMPAGNE ARDENNE LORRAINE' 'AQUITAINE LIMOUSIN POITOU CHARENTES'
 'AUVERGNE RHONE ALPES' 'BOURGOGNE FRANCHE COMTE' 'BRETAGNE'
 'CENTRE VAL DE LOIRE' 'CORSE' 'ILE DE FRANCE'
 'LANGUEDOC ROUSSILLON MIDI PYRENEES' 'NORD PAS DE CALAIS PICARDIE'
 'NORMANDIE' 'PAYS DE LA LOIRE' "PROVENCE ALPES COTE D'AZUR"
 'FRANCE METROPOLITAINE' 'GUADELOUPE ' 'MARTINIQUE ' 'GUYANE' 'LA REUNION'
 'MAYOTTE' 'DOM' 'FRANCE METROPOLITAINE ET DOM' 'NAN'
 'SOURCE : INSEE   ESTIMATIONS DE POPULATION (RESULTATS PROVISOIRES ARRÊTES FIN 2015).']
                                         dep_mon  Effectif  0 à 19 ans  \
REGION                                                                   
MAYOTTE                                 19410.51        55   2101064.0   
BRETAGNE                              1776413.88      4004  13463065.0   
GUYANE                                 114329.31       166   1831104.0   
NORMANDIE                             3543942.20      3873  13923289.0   
PAYS DE LA LOIRE          

C:\Users\Mohamed\Anaconda3\lib\site-packages\ipykernel\__main__.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
